In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
import yaml

sys.path.append("./")
from Finance.Data import Loader

# Load data
loader = Loader("data/")
data, info = loader.load("000300")
print(info)

# Load Network Config
an_config = yaml.load(open("an_config.yaml"))
print(an_config)
cn_config = yaml.load(open("cn_config.yaml"))
print(cn_config)

{'level': 'minute', 'info': '沪深300指数', 'end': '2016-12-30', 'file': '000300.pkl', 'start': '2007-01-04', 'source': '米筐量化平台'}
{'batch_size': 100, 'h0': 128, 'h1': 128, 'hidden_layer': 2, 'state_size': 10, 'tau': 0.01, 'learning_rate': 0.0001, 'action_size': 3}
{'batch_size': 100, 'h0': 128, 'h1': 128, 'hidden_layer': 3, 'state_size': 10, 'h2': 128, 'tau': 0.01, 'learning_rate': 0.0001, 'action_size': 3}


In [3]:
# Build Actor-Critic Model
from Finance.PolicyNetwork import ActorNetwork, CriticNetwork

# Optimize GPU
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

# Build Network
actor = ActorNetwork(sess, an_config)
critic = CriticNetwork(sess, cn_config)

actor.describe()
critic.describe()

Using TensorFlow backend.


Build ActorNetwork


/Users/FC/Documents/MasterProject/trader/Finance/PolicyNetwork.py:61: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("Ac..., inputs=Tensor("St...)`
  model = Model(input=State, output=Action)


Build ActorNetwork
Build CriticNetwork
Build CriticNetwork


/Users/FC/Documents/MasterProject/trader/Finance/PolicyNetwork.py:110: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  h2 = merge([h1, ha1], mode='sum', name="Hidden_Layer1")
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/Users/FC/Documents/MasterProject/trader/Finance/PolicyNetwork.py:114: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("Q_..., inputs=[<tf.Tenso...)`
  model = Model(input=[State, Action], output=Q)


ActorNetwork:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
State_Input (InputLayer)     (None, 10)                0         
_________________________________________________________________
Hidden_Layer0 (Dense)        (None, 128)               1408      
_________________________________________________________________
Hidden_Layer1 (Dense)        (None, 128)               16512     
_________________________________________________________________
Action_Output (Dense)        (None, 3)                 387       
Total params: 18,307
Trainable params: 18,307
Non-trainable params: 0
_________________________________________________________________
CriticNetwork:
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
State_Input (InputLayer)         (None,